In [173]:
import pandas as pd
import numpy as np
import time
import datetime
import math

In [2]:
zone = pd.DataFrame.from_csv("detector_lane_inventory.tsv", sep="\t")

In [403]:
event = pd.read_csv('trafficConditions.csv').drop(['Unnamed: 0'], axis=1)

In [331]:
event.head()

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,0,0,0,0,0,0,0,0,3,5,7,9
1,0,0,0,0,0,0,0,0,0,0,5,3
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
sampleZone = zone[(zone['zone_id'] == 1160) | (zone['zone_id'] == 3232) | (zone['zone_id'] == 3445) | (zone['zone_id'] == 3451) | (zone['zone_id'] == 3532)].reset_index()

In [6]:
zonedf = sampleZone[['lane_id','zone_id','longitude', 'latitude']]

In [7]:
zonedf.head(2)

,lane_id,zone_id,longitude,latitude
0,490,1160,-77.446079,38.838402
1,491,1160,-77.446113,38.838421


In [8]:
trials = pd.read_csv('prediction_trials.tsv', header=0, sep='\t')


In [9]:
trials.head(2)

,trial_id,nw_lat,nw_lon,se_lat,se_lon,start,end
0,p_00000001,39.182985,-76.798663,39.170238,-76.778150,2015-02-01T00:00:00-05:00,2015-03-01T00:00:00-05:00
1,p_00000002,39.303607,-76.829335,39.296709,-76.806935,2016-01-08T00:00:00-05:00,2016-01-22T00:00:00-05:00


In [10]:
mypoint = zonedf.as_matrix()
pts = mypoint[:, [2, 3]]

In [11]:
start_time = time.time()

ss = []

for tin in range(len(trials)):
    
    x1, x2 = trials.loc[tin]['nw_lon'], trials.loc[tin]['se_lon']
    y1, y2 = trials.loc[tin]['se_lat'], trials.loc[tin]['nw_lat']


    ll = np.array([x1, y1])  # lower-left
    ur = np.array([x2, y2])  # upper-right

    inidx = np.all(np.logical_and(ll <= pts, pts <= ur), axis=1)
    #inbox = pts[inidx]
    pos = np.where(inidx==True)
    if(len(pos[0]) > 0):
        #print mypoint[pos]
        ss.append((tin, mypoint[pos][0][1]))
    
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 195.1862461566925 seconds ---


In [408]:
sampleZone = pd.DataFrame(index=range(len(ss)))

In [409]:
sampleZone['zoneid'] = pd.DataFrame.from_dict(ss)[1].astype(int)

In [410]:
sampleTrial = trials.iloc[pd.DataFrame.from_dict(ss)[0].tolist(),:].reset_index()
sampleEvent = event.iloc[pd.DataFrame.from_dict(ss)[0].tolist(),:].reset_index()

In [407]:
sampleTrial.to_csv('sampleTrial.csv')
sampleEvent.to_csv('sampleEvent.csv')
sampleZone.to_csv('sampleZone.csv')

In [312]:
sampleTrial = pd.read_csv('sampleTrial.csv').drop(['Unnamed: 0'], axis=1)
sampleEvent = pd.read_csv('sampleEvent.csv').drop(['Unnamed: 0'], axis=1)
sampleZone = pd.read_csv('sampleZone.csv').drop(['Unnamed: 0'], axis=1)

In [411]:
z3232 = pd.DataFrame.from_csv('3232.txt', sep="\t").reset_index()
z3232['year'] = z3232['year'].astype(str).str[:4].astype(int)

In [412]:
z3445= pd.DataFrame.from_csv('3445.txt', sep="\t").reset_index()
z3445['year'] = z3445['year'].astype(str).str[:4].astype(int)

In [413]:
z3451 = pd.DataFrame.from_csv('3451.txt', sep="\t").reset_index()
z3451['year'] = z3451['year'].astype(str).str[:4].astype(int)

In [414]:
z3451 = pd.concat([z3451, pd.DataFrame(z3451.mean()).T, pd.DataFrame(z3451.mean()).T])\
.reset_index().drop('index', axis=1)

In [415]:
z3451['year'].iloc[6] = 2015
z3451['year'].iloc[7] = 2016

In [416]:
z3532 = pd.DataFrame.from_csv('3532.txt', sep="\t").reset_index()
z3532['year'] = z3532['year'].astype(str).str[:4].astype(int)

In [417]:
year = np.array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]).reshape(-1,1)
event = sampleEvent.drop('index', axis=1).as_matrix()

In [418]:
eventmatrix = event[:, 0:11]
event2014 = sampleEvent['2014'].as_matrix()

In [419]:
from sklearn.linear_model import Ridge

In [420]:
predict = []
error = []

for i in range(len(sampleTrial)):
    if(sampleZone['zoneid'][i]==3232):
        zoneMatrix = z3232.as_matrix()[:, :]
    
    if(sampleZone['zoneid'][i]==3445):
        zoneMatrix = z3445.as_matrix()[:, :]
        
    if(sampleZone['zoneid'][i]==3451):
        zoneMatrix = z3451.as_matrix()[:, :]
        
    if(sampleZone['zoneid'][i]==3532):
        zoneMatrix = z3532.as_matrix()[:, :]
        
    features = np.concatenate((year, np.concatenate((np.zeros((6,3)), zoneMatrix[:,1:])) ), axis=1)
    xtrain = features[:-3,:]
    ytrain = eventmatrix[i]
    xtest = features[-3:-2,:]
    ytest = event2014[i]
    
    #if(np.count_nonzero(ytrain) > 5):
        #clf = Ridge(alpha=15)
    #else:
    clf = Ridge(alpha=1)
    
    clf.fit(xtrain, ytrain)    
    predict.append(clf.predict(xtest)[0])
    error.append(((clf.predict(xtest)[0] - ytest) ** 2))
    

In [327]:
errA = np.sqrt(np.sum(error)/len(error))
errA

143.92830646202239

In [353]:
errD = np.sqrt(np.sum(error)/len(error))
errD

50.108215792420694

In [368]:
errO = np.sqrt(np.sum(error)/len(error))
errO

10.989780425778843

In [387]:
errP = np.sqrt(np.sum(error)/len(error))
errP

0.17648032380147063

In [402]:
errR = np.sqrt(np.sum(error)/len(error))
errR

21.252044399915214

In [421]:
errT = np.sqrt(np.sum(error)/len(error))
errT

2.3589082363499254

In [422]:
np.sqrt((errA**2 + errD**2 + errO**2 + errP**2 + errR**2 + errT**2)/6)

62.987050189279174

In [ ]:
np.sqrt((errA**2 + errD**2 + errO**2 + errP**2 + errR**2 + errT**2)/6)/12